# 02 GDP Growth and Recession Label

Compute GDP growth variants and build a technical recession label.


## Table of Contents
- Fetch GDP
- Compute growth
- Define recession label
- Define next-quarter target


## Why This Notebook Matters
Data notebooks build the datasets used everywhere else. If these steps are wrong, every model result is suspect.
You will practice:
- API ingestion and caching,
- frequency alignment,
- label construction.


## What You Will Produce
- data/processed/gdp_quarterly.csv

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.
- You can point to the concrete deliverable(s) listed below and explain how they were produced.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Merging mixed-frequency series without explicit resampling/aggregation.
- Forgetting to shift targets for forecasting tasks.

## Matching Guide
- `docs/guides/01_data/02_gdp_growth_and_recession_label.md`



## How To Use This Notebook
- This notebook is hands-on. Most code cells are incomplete on purpose.
- Complete each TODO, then run the cell.
- Use the matching guide (`docs/guides/01_data/02_gdp_growth_and_recession_label.md`) for deep explanations and alternative examples.
- Write short interpretation notes as you go (what changed, why it matters).



## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.


In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Compute GDP growth (QoQ, annualized, YoY) and define a technical recession label from GDP growth.


## Your Turn: Load GDP


In [ ]:
# TODO: Fetch GDPC1 from FRED and build a quarterly DataFrame
# Hint: convert dates to quarter-end timestamps
...


## Your Turn: Compute Growth Variants


In [ ]:
from src import macro

# TODO: Compute qoq, qoq_annualized, yoy growth
...


## Your Turn: Technical Recession Label


In [ ]:
# TODO: Implement recession_t = 1 if growth_t < 0 and growth_{t-1} < 0
# Then define target_recession_next_q = recession.shift(-1)
...


## Reflection


- Why is this only a proxy for 'recession'?
- How could this differ from an official recession indicator?


## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
import pandas as pd

# Expected file: data/processed/gdp_quarterly.csv
# TODO: After saving your processed dataset, load it and run checks.
# df = pd.read_csv(PROCESSED_DIR / 'your_file.csv', index_col=0, parse_dates=True)
# assert df.index.is_monotonic_increasing
# assert df.shape[0] > 20
# print(df.dtypes)
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Fetch GDP</summary>

```python
import pandas as pd
from src import data as data_utils

# Offline default
gdp = pd.read_csv(SAMPLE_DIR / 'gdp_quarterly_sample.csv', index_col=0, parse_dates=True)
gdp.head()
```

</details>

<details><summary>Solution: Compute growth</summary>

```python
from src import macro

levels = gdp['GDPC1']
gdp['gdp_growth_qoq'] = macro.gdp_growth_qoq(levels)
gdp['gdp_growth_qoq_annualized'] = macro.gdp_growth_qoq_annualized(levels)
gdp['gdp_growth_yoy'] = macro.gdp_growth_yoy(levels)
```

</details>

<details><summary>Solution: Define recession label</summary>

```python
gdp['recession'] = macro.technical_recession_label(gdp['gdp_growth_qoq'])
```

</details>

<details><summary>Solution: Define next-quarter target</summary>

```python
gdp['target_recession_next_q'] = macro.next_period_target(gdp['recession'])
gdp = gdp.dropna()
gdp.tail()
```

</details>

